In [1]:
from tensorflow.keras.layers import Input, Conv2D, Dense, ReLU, Flatten, Softmax
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import json

2024-04-12 17:57:35.976244: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 17:57:36.045597: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 17:57:37.177274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
temp = []
for i in range(len(y_train)):
    temp.append(to_categorical(y_train[i], num_classes=10))
y_train = np.array(temp)
temp = []
for i in range(len(y_test)):    
    temp.append(to_categorical(y_test[i], num_classes=10))
y_test = np.array(temp)

In [4]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(1,3)(inputs)
out = ReLU()(out)
out = Flatten()(out)
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 1)         10        
                                                                 
 re_lu (ReLU)                (None, 26, 26, 1)         0         
                                                                 
 flatten (Flatten)           (None, 676)               0         
                                                                 
 dense (Dense)               (None, 10)                6770      
                                                                 
 softmax (Softmax)           (None, 10)                0         
                                                                 
Total params: 6,780
Trainable params: 6,780
Non-trainable par

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 6.4538 - acc: 0.8236 - val_loss: 1.1087 - val_acc: 0.8525
Epoch 2/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.5000 - acc: 0.8917 - val_loss: 0.3109 - val_acc: 0.9121
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3086 - acc: 0.9133 - val_loss: 0.2980 - val_acc: 0.9155
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3054 - acc: 0.9142 - val_loss: 0.2854 - val_acc: 0.9203
Epoch 5/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3044 - acc: 0.9151 - val_loss: 0.3107 - val_acc: 0.9121
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3013 - acc: 0.9160 - val_loss: 0.2802 - val_acc: 0.9229
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2936 - acc: 0.9181 - val_loss: 0.3004 - val_acc: 0.9191
Epoch 8/10
1875/1875 [==============================] -

In [9]:
X = X_test[0]
X.shape

(28, 28, 1)

In [10]:
model2 = Model(model.input, model.layers[-2].output)

In [11]:
y = model2.predict(X_test[[0]]) - model.weights[3].numpy()

1/1 [==============================] - 0s 79ms/step


In [12]:
input_json = {
    "in": X.astype(int).flatten().tolist(),
    "conv2d_weights": (model.weights[0].numpy()*(10**9)).round().astype(int).flatten().tolist(),
    "conv2d_bias": (model.weights[1].numpy()*(10**9)).round().astype(int).flatten().tolist(), # no need to sqaure the scaling because input was not scaled
    "dense_weights":(model.weights[2].numpy()*(10**9)).round().astype(int).flatten().tolist(),
    "dense_bias": np.zeros(model.weights[3].numpy().shape).tolist() # zero because we are not doing softmax in circom, just argmax
}

In [13]:
output_json = {
    "scale": 10**-18,
    "out": y.flatten().tolist(),
    "label": int(y.argmax())
}

In [14]:
with open("CNN1_input.json", "w") as file:
    json.dump(input_json, file)

In [15]:
with open("CNN1_output.json", "w") as file:
    json.dump(output_json, file)